In [3]:
import pandas as pd
import numpy as np
from os import listdir
from os import path
import prepare_drafts as drafts


def create_lookup(df: pd.DataFrame, col: str, lookup_type: str) -> pd.DataFrame:
    """ Creates a section of the lookups data. """
    df = df.copy()
    
    keep_vars = ['draft_year', col]
    df = df_w_rank[keep_vars].drop_duplicates(subset=keep_vars)

    df.rename(columns={col: 'drafts_val'}, inplace=True)
    df['ranks_val'] = df['drafts_val']
    df['lookup_type'] = lookup_type

    df = df.loc[df['drafts_val'].notnull()]

    df = df[['lookup_type', 'draft_year', 'drafts_val', 'ranks_val']]

    return df


def export_lookups(df: pd.DataFrame, folder_path: str, file_name: str) -> None:
    df_player = create_lookup(df, 'full_name', 'player')
    df_team = create_lookup(df, 'team_name', 'team')
    df_pos = create_lookup(df, 'position', 'position')

    df = pd.concat([df_player, df_team, df_pos])

    df.to_csv(path.join(folder_path, file_name), index=False)


pd.set_option('display.max_rows', 300)

DRAFTS_FOLDER = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model/data'

df_drafts = drafts.read_raw_data(DRAFTS_FOLDER, [2021])
df_updated_types = drafts.update_dtypes(df_drafts)
df_complete_players = drafts.drafts_w_player_data(df_updated_types)
df_draft_attrs = drafts.add_draft_attrs(df_complete_players)
df_w_rank = drafts._add_rank_derived(df_draft_attrs)

Index(['appearance_id', 'created_at', 'draft_entry_id', 'number',
       'projection_adp', 'draft_id', 'position', 'team_name', 'draft_source',
       'draft_title', 'full_name', 'num_teams', 'round', 'round_pick',
       'draft_datetime', 'draft_date', 'draft_year', 'rank_derived'],
      dtype='object')


,appearance_id,created_at,draft_entry_id,number,projection_adp,draft_id,position,team_name,draft_source,draft_title,full_name,num_teams,round,round_pick,draft_datetime,draft_date,draft_year,rank_derived
11664,78a5634d-93aa-4cd9-b1af-dfb829df452c,2021-05-12 23:06:31,83d437dd-902c-418c-a05b-e7865808398e,1,1.1,d8ef2c43-a686-4854-a72b-5929ab895ec1,RB,Carolina Panthers,NaN,NaN,Christian McCaffrey,12,1,1,2021-05-12 23:06:31,2021-05-12,2021,1
11665,a971451b-f89e-498f-a6b4-060854880ed6,2021-05-12 23:06:33,53b6ad26-7f17-4a54-b065-ccfea7772503,2,2.1,d8ef2c43-a686-4854-a72b-5929ab895ec1,RB,Minnesota Vikings,NaN,NaN,Dalvin Cook,12,1,2,2021-05-12 23:06:31,2021-05-12,2021,2
11666,53565b50-b52d-4c4f-bcf7-ee96476f4bbc,2021-05-12 23:06:50,7e54d614-015d-47d2-a77b-2bbce63bb9bf,3,3.9,d8ef2c43-a686-4854-a72b-5929ab895ec1,RB,Tennessee Titans,NaN,NaN,Derrick Henry,12,1,3,2021-05-12 23:06:31,2021-05-12,2021,4
11667,fbaa93f9-6961-407f-9042-5d033198e870,2021-05-12 23:07:02,277ba9f9-9ae9-45ac-b5ac-b0a3b3aa0b10,4,3.5,d8ef2c43-a686-4854-a72b-5929ab895ec1,RB,NY Giants,NaN,NaN,Saquon Barkley,12,1,4,2021-05-12 23:06:31,2021-05-12,2021,3
11668,30149020-3a70-47ce-a8e2-b52f0a5770fa,2021-05-12 23:07:13,1e2cf73f-982b-4444-8df7-44e8c0ccbab0,5,5.4,d8ef2c43-a686-4854-a72b-5929ab895ec1,RB,New Orleans Saints,NaN,NaN,Alvin Kamara,12,1,5,2021-05-12 23:06:31,2021-05-12,2021,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18355,5b713430-bbe6-4e7d-94b3-a8d1234e5d44,2021-09-04 01:45:49,f32698c7-99c9-422c-b59b-11fc7f90789a,212,201.1,f75ed573-6a11-4e59-b712-1e8826d05c44,RB,New Orleans Saints,sit_and_go,NaN,Tony Jones,12,18,8,2021-09-04 01:01:06,2021-09-04,2021,197
18356,54fe78c0-ea25-423b-8b9d-e75d60ac6f83,2021-09-04 01:45:53,ec17eb86-2034-4a16-bd78-9fd91314da4f,213,203.4,f75ed573-6a11-4e59-b712-1e8826d05c44,WR,Green Bay Packers,sit_and_go,NaN,Allen Lazard,12,18,9,2021-09-04 01:01:06,2021-09-04,2021,200
18357,c7b76231-b0f7-4156-ab5b-b6f3f9511760,2021-09-04 01:46:06,1b3250f1-8553-4b91-8863-7fb08909a5ac,214,214.9,f75ed573-6a11-4e59-b712-1e8826d05c44,TE,Indianapolis Colts,sit_and_go,NaN,Mo Alie-Cox,12,18,10,2021-09-04 01:01:06,2021-09-04,2021,215
18358,f9bfb56d-bb33-4c6a-925f-87b504efc696,2021-09-04 01:46:29,e533cd03-0e2a-40b0-9ea6-cd3f67d4bc26,215,213.6,f75ed573-6a11-4e59-b712-1e8826d05c44,TE,Buffalo Bills,sit_and_go,NaN,Dawson Knox,12,18,11,2021-09-04 01:01:06,2021-09-04,2021,212


In [18]:
##############################################################################################
############################# Create Custom Lookups ##########################################
##############################################################################################

OUTPUT_FOLDER = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model\
/data/2021'
export_lookups(df_w_rank, OUTPUT_FOLDER, 'lookups_2021.csv')


In [25]:
##############################################################################################
############################### Create Custom Ranks ##########################################
##############################################################################################

keep_vars = ['full_name', 'position', 'team_name', 'projection_adp'
            , 'rank_derived', 'draft_date']
rename_vars = {'full_name': 'player', 'position': 'pos', 'team_name': 'team'
            , 'projection_adp': 'adp', 'rank_derived': 'rank', 'draft_date': 'date'}

df = df_w_rank[keep_vars].copy()
df.rename(columns=rename_vars, inplace=True)

OUTPUT_FOLDER = '/home/cdelong/Python-Projects/UD-Draft-Model/Repo-Work/UD-Draft-Model\
/data/2021/player_ranks'
df.to_csv(path.join(OUTPUT_FOLDER, 'df_player_ranks_2021.csv'), index=False)